# This notebook is just straight up dev of the target inferring code

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np
import sumolib

In [4]:
from utils.data_utils.network_utils import (flatten,
                                            get_downstream_edges,
                                            get_upstream_edges,
                                            get_edge_to_level_dict, 
                                            get_up_and_down_stream)
from utils.result_utils.plot_utils import plotNet
from utils.result_utils.file_utils import xml2csv_path
from utils.result_utils.dotdict import DotDict
from utils.general_utils.conversion_utils import convert_seconds_to_24h
from utils.data_utils.preprocess_utils import (transform_df_to_tensor,
                                                get_index_to_edge_dicts,
                                                idxs_of_longest_seq,
                                               find_congestion,
                                               infer_incident_data)
from utils.general_utils.conversion_utils import trig_transform

In [5]:
scenario = 'motorway'
experiment_name = 'incident3_fix_test_0'
#path = f'../{scenario}/Results/{experiment_name}'
path = '../motorway/Results/incident3/incident3_729'


In [6]:
result_folders = os.listdir(path)
print(f'{len(result_folders)} simulations')
experiment_folders = result_folders

11 simulations


In [7]:
net_path = '../motorway/Simulations/Base/network.net.xml'
net = sumolib.net.readNet(net_path)

In [9]:
# 16 shows not smooth enough
#experiment = experiment_folders[0]
#experiment_path = f'{path}/{experiment}'
#experiment_path = f'{path}/incident1_257'
experiment_path = path
print(experiment_path)
#os.listdir(experiment_path)

with open(f'{experiment_path}/incident_settings.json') as f:
    incident_settings = json.load(f)
print(incident_settings)

incident_edge = incident_settings['edge']
i_edge_obj = net.getEdge(incident_edge)

hours, minutes, seconds = convert_seconds_to_24h(incident_settings['start_time'])
print(f'Incident at {hours}:{minutes}:{seconds}')



../motorway/Results/incident3/incident3_729
{'slow_zone': 70, 'lc_zone': 20, 'lc_prob_zone': 170, 'slow_zone_speed': 13.8, 'run_num': 729, 'is_incident': True, 'is_random': True, 'random_seed': 1994322105, 'edge': '328393125', 'lanes': [1, 2, 3], 'pos': 27.945571553555354, 'start_time': 81121, 'start_step': 162242, 'duration_time': 1269, 'duration_steps': 2538}
Incident at 22:32:1


In [10]:
edge_to_level_dict, upstream_edges, downstream_edges, relevant_edges = get_up_and_down_stream(i_edge_obj=i_edge_obj,
                                                                                              n_up=40,
                                                                                              n_down=40)

79921.0

In [52]:
inter_lim = 2
inci_df = pd.read_csv(f'{experiment_path}/detectordata.csv', sep=';')
counter_df = pd.read_csv(f'{experiment_path}/detectordata_counterfactual.csv', sep=';')

min_inci_time = inci_df.interval_begin.min()
max_inci_time = inci_df.interval_begin.max()
min_counter_time = counter_df.interval_begin.min()
max_counter_time = counter_df.interval_begin.max()
min_time = np.max([min_inci_time, min_counter_time])
max_time = np.min([max_inci_time, max_counter_time])
counter_df = counter_df.loc[counter_df.interval_begin.between(min_time,max_time)]
inci_df = inci_df.loc[inci_df.interval_begin.between(min_time,max_time)]


inci_data, time_sequence = transform_df_to_tensor(inci_df, interpolation_lim=inter_lim, warmup=10)
counter_data, time_seqence2 = transform_df_to_tensor(counter_df, interpolation_lim=inter_lim, warmup=10)

In [58]:



# Get dicts to go from edge id to idx
ind_to_edge, edge_to_ind = get_index_to_edge_dicts(inci_df)

# Create lists of up and downstream idxs
upstream_edge_idxs = [edge_to_ind[edge] for edge in upstream_edges]
downstream_edge_idxs = [edge_to_ind[edge] for edge in downstream_edges]

# Mask out padded lanes
lane_mask = ~(inci_data[...,1].mean(-1) == -1)

# Mask out lanes that are not used during normal traffic
unused_lanes_mask = ((counter_data[...,2].sum(2) > 0) & (counter_data[...,2].sum(2) < 300))
inci_data[unused_lanes_mask] = -2
counter_data[unused_lanes_mask] = -2

# Create residual data
residual_data = inci_data - counter_data[:, :, :,:]

length_lim = 10
std_lim = 1.95

# Get normal condition lane STD
lane_stds = counter_data[...,1].std(2)

# Only look at speed
residual_speed = residual_data[...,1]

# Bool of affected lane at time steps [E, L ,T]
affect_bool_arr = (residual_speed.transpose(2,0,1) < std_lim * -lane_stds)

# reshape for finding consecutive subsequences -> [E*L, T]
arr = affect_bool_arr.transpose(1,2,0).reshape(-1, 231)

# Find longest subsequences
longest_affect_idxs, longest_affect_lengths = zip(*[idxs_of_longest_seq(row) for row in arr])

# Reshape back
longest_affect_lengths = np.stack(longest_affect_lengths).reshape(147,6)
longest_affect_idxs = np.stack(longest_affect_idxs).reshape(147,6,2)

# Find edges where a lane has a subsequence longer than limit
affected_edges = (longest_affect_lengths > length_lim).any(axis=1)
edge_idxs = np.where(affected_edges)[0]
affected_edges.sum()


ValueError: cannot reshape array of size 92610 into shape (231)

In [ ]:
upstream_edges_arr = np.isin(np.arange(147), upstream_edge_idxs)
affected_us_mask = (upstream_edges_arr * affected_edges)

affected_upstream_edge_idxs = np.where(affected_us_mask)[0]

affected_us_mask.sum()


In [ ]:
residual_speed.shape

In [ ]:
lane_stds.shape

In [ ]:
length_lim_2 = 1
std_lim_2 = 0.5

# Bool of affected lane at time steps [E, L ,T]
affect_bool_arr_2 = (residual_speed.transpose(2,0,1) < std_lim_2 * -lane_stds)

# reshape for finding consecutive subsequences -> [E*L, T]
arr_2 = affect_bool_arr_2.transpose(1,2,0).reshape(-1, 231)

# Find longest subsequences
longest_affect_idxs_2, longest_affect_lengths_2 = zip(*[idxs_of_longest_seq(row) for row in arr_2])

# Reshape back
longest_affect_lengths_2 = np.stack(longest_affect_lengths_2).reshape(147,6)
longest_affect_idxs_2 = np.stack(longest_affect_idxs_2).reshape(147,6,2)

# Find edges where a lane has a subsequence longer than limit
affected_edges_2 = (longest_affect_lengths_2 > length_lim).any(axis=1)
edge_idxs_2 = np.where(affected_edges_2)[0]
affected_edges_2.sum()

# Take out start and end idxs
cong_start_idxs = longest_affect_idxs_2[...,0]
cong_end_idxs = longest_affect_idxs_2[...,1]

# Fill in fillers for the idxs that does not correspond to a long enough sequence
cong_start_idxs[(longest_affect_lengths < length_lim)] = np.inf
cong_end_idxs[(longest_affect_lengths < length_lim)] = -np.inf

# For each edge take the earliest start and latest end. 
cong_start_time = cong_start_idxs.min(1)
cong_end_time = cong_end_idxs.max(1)

# Mask out unaffected edges
cong_start_time[~affected_us_mask] = 0
cong_end_time[~affected_us_mask] = 0

# Calculate the max speed decrease
delta_speeds = np.zeros((147))
for edge in affected_upstream_edge_idxs:
    start_time = int(cong_start_time[edge])
    end_time = int(cong_end_time[edge])
    #counter_mean_speed = counter_data[edge, lane_mask[edge], start_time:end_time, 2].mean(-1)
    #inci_mean_speed = inci_data[edge, lane_mask[edge], start_time:end_time, 2].mean(-1)
    #delta_speeds[edge, lane_mask[edge]] = counter_mean_speed - inci_mean_speed
    delta_speeds[edge] = residual_speed[edge, lane_mask[edge], start_time:end_time].min()
    
secs_in_day = 24 * 60 * 60


trig_time = np.array([trig_transform(seconds, secs_in_day) for seconds in time_sequence])

In [ ]:
incident_time = np.where(time_sequence == incident_settings['start_time'])[0].item()
input_traffic_data = inci_data[:,:,:incident_time, :]
input_time = np.expand_dims(trig_time[:input_traffic_data.shape[2]], axis=(0,1))
input_time = np.repeat(input_time, input_traffic_data.shape[0], axis=0)
input_time = np.repeat(input_time, input_traffic_data.shape[1], axis=1)
input_data = np.concatenate([input_traffic_data, input_time], axis=-1)

target_data = np.stack([affected_us_mask, cong_start_time, cong_end_time, delta_speeds], axis=-1)

In [ ]:
inci_data.shape

In [ ]:
input_data.shape

In [ ]:
target_data.shape

In [ ]:
input_data2, target_data2, longest_affect_lengths2, longest_affect_idxs2 = infer_incident_data(experiment_path, scenario='motorway')


In [ ]:
longest_affect_lengths_2 - longest_affect_lengths2

In [ ]:
(input_data - input_data2).sum()

In [ ]:
np.where(target_data - target_data2 != 0)

In [ ]:
target_data[75]

In [ ]:
target_data2[75]

In [ ]:
(target_data - target_data2)[75]